<a href="https://colab.research.google.com/github/lyw02/EtymologyLM/blob/main/nb/Qwen3_(4B)-GRPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

Read our **[Qwen3 Guide](https://docs.unsloth.ai/basics/qwen3-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm

In [2]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

### Unsloth

Goal: To convert `Qwen3-4B-Base` into a reasoning model via GRPO by using OpenR1's Math dataset.

We first pre fine-tune the model to make GRPO skip trying to match formatting - this speeds GRPO up.

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-14B-unsloth-bnb-4bit",
    max_seq_length = max_seq_length,
    load_in_4bit = False, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.5, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank*2, # *2 speeds up training
    use_gradient_checkpointing = "unsloth", # Reduces memory usage
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-17 15:11:58 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-17 15:11:58 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.5.6: Fast Qwen3 patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/Qwen3-8B-Base with actual GPU utilization = 49.43%
Unsloth: Your GPU has CUDA compute capability 8.0 with VRAM = 39.56 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill t

tokenizer_config.json:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Unsloth 2025.5.6 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


### GRPO chat template
Since we're using a base model, we should set a chat template. You can make your own chat template as well!
1. DeepSeek uses `<think>` and `</think>`, but this is **not** necessary - you can customize it however you like!
2. A `system_prompt` is recommended to at least guide the model's responses.

In [4]:
structure_start = "<structure>"
structure_end   = "</structure>"
content_start   = "<content>"
content_end     = "</content>"

system_prompt = \
f"""You are a helpful linguist, especially expertise in English etymology.
The user will give you a piece of text that describes etymology of a word, you must output its structured etymology information in the given pattern.
Always output ONLY the structured information. No extra words.
Always remember, in side the output, etymology structure should be placed between {structure_start} and {structure_end}, and detailed content of each node should be placed between {content_start} and {content_end}.
If the input is not relevant to etymology, you should output CANNOT_RECOGNISE."""
system_prompt

'You are a helpful linguist, especially expertise in English etymology.\nThe user will give you a piece of text that describes etymology of a word, you must output its structured etymology information in the given pattern.\nAlways output ONLY the structured information. No extra words.\nAlways remember, in side the output, etymology structure should be placed between <structure> and </structure>, and detailed content of each node should be placed between <content> and </content>.\nIf the input is not relevant to etymology, you should output CANNOT_RECOGNISE.'

We create a simple chat template below. Notice `add_generation_prompt` includes prepending `<start_working_out>` to guide the model to start its reasoning process.

In [5]:
chat_template = \
    "{{ '{system_prompt}' + eos_token }}"\
    "{% for message in messages[1:] %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ message['content'] }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ message['content'] + eos_token }}"\
        "{% endif %}"\
    "{% endfor %}"

# Replace with out specific template:
chat_template = chat_template\
    .replace("'{system_prompt}'",   f"'{system_prompt}'")
tokenizer.chat_template = chat_template

Let's see how our chat template behaves on an example:

In [6]:
tokenizer.apply_chat_template([
    {"role" : "user", "content" : """
Summary
A borrowing from Latin.
Etymon: Latin formātiōn-em.
< Latin formātiōn-em, noun of action < formāre to form v.1: see ‑ation suffix.
    """},
    {"role" : "assistant", "content" : f"""
{structure_start}
English - Latin 1 - Latin 2
{structure_end}
{content_start}
@English: {{formation}}
@Latin 1: {{formātiōn-em}} [noun of action]
@Latin 2: {{formāre}} [to form v.1]
{content_end}
    """},
], tokenize = False, add_generation_prompt = True)

'You are a helpful linguist, especially expertise in English etymology.\nThe user will give you a piece of text that describes etymology of a word, you must output its structured etymology information in the given pattern.\nAlways output ONLY the structured information. No extra words.\nAlways remember, in side the output, etymology structure should be placed between <structure> and </structure>, and detailed content of each node should be placed between <content> and </content>.\nIf the input is not relevant to etymology, you should output CANNOT_RECOGNISE.<|endoftext|>\n<structure>\nEnglish - Latin 1 - Latin 2\n</structure>\n<content>\n@English: {formation}\n@Latin 1: {formātiōn-em} [noun of action]\n@Latin 2: {formāre} [to form v.1]\n</content>\n    <|endoftext|>'

### Pre fine-tuning for formatting
We now use a subset of NVIDIA's [Open Math Reasoning dataset](https://huggingface.co/datasets/nvidia/OpenMathReasoning) which was filtered to only include high quality DeepSeek R1 traces.

We'll only filter ~59 or so examples to first "prime" / pre fine-tune the model to understand our custom GRPO formatting.

In [7]:
from datasets import load_dataset
import pandas as pd
import numpy as np


dataset = load_dataset("lyw02/ety-100-core", split = "train")
df = dataset.to_pandas()
df_exp = df.explode("messages")
msg_df = pd.json_normalize(df_exp["messages"])
msg_df["conv_id"] = df_exp.index
agg = (
    msg_df
    .groupby(["conv_id", "role"])["content"]
    .apply(list)
    .unstack(fill_value=[])
)
agg["user"] = agg["user"].str[0]
agg["assistant"] = agg["assistant"].str[0]

dataset = agg.reset_index(drop=True)[["user", "assistant"]]

dataset = dataset.head(50)

dataset

annotated_100_train.jsonl:   0%|          | 0.00/225k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

role,user,assistant
0,Summary\nOf multiple origins. Partly a borrowi...,<structure>\nEnglish - Anglo-Norman & Old Fren...
1,Summary\nOf multiple origins. Partly a borrowi...,<structure>\nEnglish - Middle French\nEnglish ...
2,Summary\nA word inherited from Germanic.\nCogn...,<structure>\nEnglish - Old Frisian - Germanic\...
3,Summary\nOf multiple origins. Partly a borrowi...,<structure>\nEnglish - Anglo-Norman & Middle F...
4,Summary\nA borrowing from French.\nEtymon: Fre...,<structure>\nEnglish - Anglo-Norman - Latin 1 ...
5,Summary\nA borrowing from French.\nEtymon: Fre...,<structure>\nEnglish - Middle English - Old Fr...
6,Summary\nOf multiple origins. Partly a borrowi...,<structure>\nEnglish - Anglo-Norman & Old Fren...
7,Summary\nA borrowing from French.\nEtymon: Fre...,<structure>\nEnglish - Old French - Latin 1 - ...
8,Summary\nA borrowing from French.\nEtymon: Fre...,<structure>\nEnglish - Old French 1 - Old Fren...
9,Summary\nProbably a borrowing from French.\nNo...,<structure>\nEnglish - Latin ?- French\n</str...


We have to format the dataset to follow our GRPO style formatting:

In [8]:
def format_dataset(x):
    user = x["user"]
    assistant = x["assistant"]

    return [
        {"role" : "system",    "content" : system_prompt},
        {"role" : "user",      "content" : user},
        {"role" : "assistant", "content" : assistant},
    ]

dataset["Messages"] = dataset.apply(format_dataset, axis = 1)

Check to see if it worked:

In [9]:
tokenizer.apply_chat_template(dataset["Messages"][0], tokenize = False)

'You are a helpful linguist, especially expertise in English etymology.\nThe user will give you a piece of text that describes etymology of a word, you must output its structured etymology information in the given pattern.\nAlways output ONLY the structured information. No extra words.\nAlways remember, in side the output, etymology structure should be placed between <structure> and </structure>, and detailed content of each node should be placed between <content> and </content>.\nIf the input is not relevant to etymology, you should output CANNOT_RECOGNISE.<|endoftext|>Summary\nOf multiple origins. Partly a borrowing from French. Partly a borrowing from Latin.\nEtymons: French espirit, spirit; Latin spīritus.\n< (i) Anglo-Norman esperite, espirite, espirith, (rare) spirit, Anglo-Norman and Old French, Middle French esperit, espirit (Anglo-Norman and Middle French, French esprit) animating or vital principle, wind, breath, air, action of breathing, divine inspiration, consciousness, em

Let's truncate the pre fine-tuning dataset to `max_seq_length/2` since we don't want too long reasoning traces.

Note this might take 2 minutes!

In [10]:
dataset["N"] = dataset["Messages"].apply(lambda x: len(tokenizer.apply_chat_template(x)))

# dataset = dataset.loc[dataset["N"] <= max_seq_length/2].copy()
dataset.shape

(50, 4)

We then tokenize the messages and convert it to a Hugging Face compatible dataset format:

In [11]:
from datasets import Dataset

dataset["text"] = tokenizer.apply_chat_template(dataset["Messages"].values.tolist(), tokenize = False)
dataset = Dataset.from_pandas(dataset)
dataset

Dataset({
    features: ['user', 'assistant', 'Messages', 'N', 'text'],
    num_rows: 50
})

Let's now pre fine-tune the model so it follows our custom GRPO formatting!

In [12]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 1, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 10, # Set this for 1 full training run.
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=12):   0%|          | 0/50 [00:00<?, ? examples/s]

In [13]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 50 | Num Epochs = 10 | Total steps = 500
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 1 x 1) = 1
 "-____-"     Trainable parameters = 87,293,952/8,278,029,312 (1.05% trained)


Step,Training Loss
10,1.916400
20,0.997200
30,0.859900
40,0.885500
50,0.742700
60,0.658000
70,0.459800
80,0.707300
90,0.531700
100,0.528200


Unsloth: Will smartly offload gradients to save VRAM!


TrainOutput(global_step=500, training_loss=0.24791481637954713, metrics={'train_runtime': 292.0828, 'train_samples_per_second': 1.712, 'train_steps_per_second': 1.712, 'total_flos': 1.43682167390208e+16, 'train_loss': 0.24791481637954713})

Let's check if the model has learnt to follow the custom format:

In [14]:
text = tokenizer.apply_chat_template(
    dataset[0]["Messages"][:2],
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    temperature = 0,
    max_new_tokens = 1024,
    streamer = TextStreamer(tokenizer, skip_prompt = False),
)

dataset[0]["Messages"][:2]

You are a helpful linguist, especially expertise in English etymology.
The user will give you a piece of text that describes etymology of a word, you must output its structured etymology information in the given pattern.
Always output ONLY the structured information. No extra words.
Always remember, in side the output, etymology structure should be placed between <structure> and </structure>, and detailed content of each node should be placed between <content> and </content>.
If the input is not relevant to etymology, you should output CANNOT_RECOGNISE.<|endoftext|>Summary
Of multiple origins. Partly a borrowing from French. Partly a borrowing from Latin.
Etymons: French espirit, spirit; Latin spīritus.
< (i) Anglo-Norman esperite, espirite, espirith, (rare) spirit, Anglo-Norman and Old French, Middle French esperit, espirit (Anglo-Norman and Middle French, French esprit) animating or vital principle, wind, breath, air, action of breathing, divine inspiration, consciousness, emotion, t

[{'content': 'You are a helpful linguist, especially expertise in English etymology.\nThe user will give you a piece of text that describes etymology of a word, you must output its structured etymology information in the given pattern.\nAlways output ONLY the structured information. No extra words.\nAlways remember, in side the output, etymology structure should be placed between <structure> and </structure>, and detailed content of each node should be placed between <content> and </content>.\nIf the input is not relevant to etymology, you should output CANNOT_RECOGNISE.',
  'role': 'system'},
 {'content': 'Summary\nOf multiple origins. Partly a borrowing from French. Partly a borrowing from Latin.\nEtymons: French espirit, spirit; Latin spīritus.\n< (i) Anglo-Norman esperite, espirite, espirith, (rare) spirit, Anglo-Norman and Old French, Middle French esperit, espirit (Anglo-Norman and Middle French, French esprit) animating or vital principle, wind, breath, air, action of breathing,

Run inference

In [15]:
import re

def extract_and_format_content(input_str: str) -> str:
    """
    Extract contents in <structure> and <content> tags, and return a new formatted string
    """
    input_str = input_str.split("<|endoftext|>")[1]
    structure_match = re.search(r"<structure>(.*?)</structure>", input_str, re.DOTALL)
    s_content = structure_match.group(1) if structure_match else ""

    content_match = re.search(r"<content>(.*?)</content>", input_str, re.DOTALL)
    c_content = content_match.group(1) if content_match else ""

    new_string = f"<structure>\n{s_content}\n</structure>\n<content>\n{c_content}\n</content>"

    return new_string


from unsloth.chat_templates import get_chat_template
import json
from time import time

with open('test_set_small.jsonl', 'r', encoding='utf-8') as f_in, open('inference.jsonl', 'w', encoding='utf-8') as f_out:

    eta = []
    for line in f_in:
        data = json.loads(line)
        messages = [
            {
              "role": "system",
              "content": system_prompt
            },
            {
              "role": "user",
              "content": data['messages'][1]["content"]
            }
        ]
        # print(messages)


        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt = True, # Must add for generation
        )

        t_start = time()
        outputs = model.generate(
            **tokenizer([text], return_tensors = "pt").to("cuda"),
            max_new_tokens = 4096, # Increase for longer outputs!
            # Recommended Gemma-3 settings!
            temperature = 1.0, top_p = 0.95, top_k = 64,
        )
        t_end = time()
        eta.append(t_end - t_start)

        reply = tokenizer.batch_decode(outputs)
        # print(reply)

        formatted_reply = extract_and_format_content(reply[0])
        # print(formatted_reply)

        output = [{}, {}]
        output.append({"role": "assistant", "content": formatted_reply})
        json.dump({"messages": output}, f_out)
        f_out.write('\n')
        print("Proceeded line")

print(f"ETA: {sum(eta)/len(eta)}")

Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
ETA: 11.932231330871582


Yes it did follow the formatting! Great! Let's remove some items before the GRPO step

In [16]:
del dataset
torch.cuda.empty_cache()
import gc
gc.collect()

791

### Data Prep
<a name="Data"></a>

We're using Hugging Face's [Open R1 Math dataset](https://huggingface.co/datasets/open-r1/DAPO-Math-17k-Processed). You can also utilize OpenAI's famous [GSM8K dataset](https://huggingface.co/datasets/openai/gsm8k)

In [17]:
dataset = load_dataset("lyw02/ety-100-core", split = "train")

dataset = dataset.map(lambda x: {
    "prompt" : [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": x["messages"][1]["content"]},
    ],
    "answer": x["messages"][2]["content"],
}, remove_columns=dataset.column_names)

dataset = dataset.select(range(50, 100))
dataset

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt', 'answer'],
    num_rows: 50
})

Let's look at the first row:

In [18]:
dataset[0]["prompt"]

[{'content': 'You are a helpful linguist, especially expertise in English etymology.\nThe user will give you a piece of text that describes etymology of a word, you must output its structured etymology information in the given pattern.\nAlways output ONLY the structured information. No extra words.\nAlways remember, in side the output, etymology structure should be placed between <structure> and </structure>, and detailed content of each node should be placed between <content> and </content>.\nIf the input is not relevant to etymology, you should output CANNOT_RECOGNISE.',
  'role': 'system'},
 {'content': "Summary\nA borrowing from French.\nEtymon: French descouvrir.\n< Anglo-Norman descoverir, descovrir, descuverir, discoverir, discovrir, discuverir, discouvrir, descoverer, discoverer, descoverre, Anglo-Norman and Old French, Middle French descovrir, Anglo-Norman and Middle French descouvrir (compare Anglo-Norman decoverir, decovrir, decuvrir; Middle French, French découvrir) to expo

We create a regex format to match the reasoning sections and answers:

In [19]:
import re

# Add optional EOS token matching
structure_end_regex = r"</structure>[\s]{0,}" + \
    "(?:" + re.escape(tokenizer.eos_token) + ")?"

content_end_regex = r"</content>[\s]{0,}" + \
    "(?:" + re.escape(tokenizer.eos_token) + ")?"

match_structure = re.compile(
    rf"{structure_start}(.+?){structure_end_regex}"\
    rf"[\s]{{0,}}$",
    flags = re.MULTILINE | re.DOTALL
)

match_content = re.compile(
    rf"{content_start}(.+?){content_end_regex}"\
    rf"[\s]{{0,}}$",
    flags = re.MULTILINE | re.DOTALL
)

We verify it works:

In [20]:
str_to_match_ = """
<structure>
English - Latin 1 - Latin 2
</structure>
<content>
@English: {{formation}}
@Latin 1: {{formātiōn-em}} [noun of action]
@Latin 2: {{formāre}} [to form v.1]
</content>
"""

match_structure.findall(str_to_match_)

['\nEnglish - Latin 1 - Latin 2\n']

In [21]:
match_content.findall(str_to_match_)

['\n@English: {{formation}}\n@Latin 1: {{formātiōn-em}} [noun of action]\n@Latin 2: {{formāre}} [to form v.1]\n']

We now want to create a reward function to match the format exactly - we reward it with 3 points if it succeeds:

In [22]:
def match_format_exactly(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Match if format is seen exactly!
        if match_structure.search(response) is None: score -= 5.0
        if match_content.search(response) is None: score -= 5.0
        scores.append(score)
    return scores

If it fails, we want to reward the model if it at least follows the format partially, by counting each symbol:

In [23]:
# def match_format_approximately(completions, **kwargs):
#     scores = []
#     for completion in completions:
#         score = 0
#         response = completion[0]["content"]
#         # Count how many keywords are seen - we penalize if too many!
#         # If we see 1, then plus some points!

#         # No need to reward <start_working_out> since we always prepend it!
#         # score += 0.5 if response.count(reasoning_start) == 1 else -1.0
#         score += 0.5 if response.count(structure_end)   == 1 else -1.0
#         score += 0.5 if response.count(content_start)  == 1 else -1.0
#         score += 0.5 if response.count(content_end)    == 1 else -1.0
#         scores.append(score)
#     return scores

Finally, we want to extract the generated answer, and reward or penalize it! We also reward it based on how close the answer is to the true one via ratios:

In [24]:
!pip install networkx

import networkx as nx
from networkx.algorithms import isomorphism
import re
from collections import defaultdict


def parse_structure(input_string):
    """
    Parse graph structure from a string.
    Input example:
    A - B - (C + D)
    C - E
    Returns nodes set and edges list
    """
    def parse_line(line):
        # 分割字符串，捕获分隔符
        pattern = r'(\s-\s|\s-\S+-\s)'
        parts = re.split(pattern, line.strip())
        edges = []
        i = 0
        while i < len(parts) - 2:  # 确保有源节点、分隔符、目标节点
            source = parts[i].strip()
            target = parts[i + 2].strip()
            if target.startswith('(') and target.endswith(')'):
                # 处理子图
                subgraph_str = target[1:-1].strip()  # 移除括号
                subgraph_nodes = [node.strip() for node in subgraph_str.split(' + ')]
                for node in subgraph_nodes:
                    edges.append((node, source))  # 反转方向
            else:
                # 普通连接
                edges.append((target, source))  # 反转方向
            i += 2  # 跳到下一个源节点
        return edges

    # 分割输入字符串为行
    lines = [line.strip() for line in input_string.splitlines() if line.strip()]
    all_edges = []
    for line in lines:
        edges = parse_line(line)
        all_edges.extend(edges)

    # 提取节点
    nodes = set()
    for edge in all_edges:
        nodes.add(edge[0])  # 源节点
        nodes.add(edge[1])  # 目标节点

    return nodes, all_edges


def build_graph(nodes, edges):
    G = nx.DiGraph()
    for node in nodes:
        G.add_node(node, label=node)
    for edge in edges:
        if len(edge) == 2:
            G.add_edge(edge[0], edge[1])
        elif len(edge) == 3:
            G.add_edge(edge[0], edge[1], **edge[2])
    return G


def compute_f1_score(gt_str, inf_str):
  nodes1, edges1 = parse_structure(gt_str)
  g1 = build_graph(nodes1, edges1)
  nodes2, edges2 = parse_structure(inf_str)
  g2 = build_graph(nodes2, edges2)

  # get edge sets
  edges_g1 = set(g1.edges())
  edges_g2 = set(g2.edges())

  # compute common edge set
  common_edges = edges_g1.intersection(edges_g2)

  # compute precision
  if len(edges_g2) > 0:
      precision = len(common_edges) / len(edges_g2)
  else:
      precision = 0.0  # avoid divided by 0

  # compute recall
  if len(edges_g1) > 0:
      recall = len(common_edges) / len(edges_g1)
  else:
      recall = 0.0  # avoid divided by 0

  # conpute score
  if precision + recall > 0:
      f1_score = 2 * (precision * recall) / (precision + recall)
  else:
      f1_score = 0.0

  return f1_score

In [25]:
def check_answer(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_structure.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    for guess, true_answer in zip(extracted_responses, answer):
        score = 0
        if guess is None:
            scores.append(-2.0)
            continue

        true_answer = match_structure.findall(true_answer)[0]

        # F1 score reward
        score += (compute_f1_score(true_answer, guess) * 10 - 5)

        # nodes_true_answer, _ = parse_structure(true_answer)
        # nodes_guess, _ = parse_structure(guess)

        scores.append(score)
    return scores

In [26]:
def check_node_label(prompts, completions, **kwargs):
    responses = [c[0]["content"] for c in completions]
    extracted = [
        m.group(1) if (m := match_structure.search(r)) else None
        for r in responses
    ]

    scores = []
    for guess in extracted:
        if guess is None:
            scores.append(-2.0)
            continue

        nodes, _ = parse_structure(guess)
        counts = defaultdict(int)
        nums   = defaultdict(set)
        node_pat = re.compile(r'^(?P<name>[^\d ].*?)(?: (?P<num>\d+))?$')

        score = 0.0
        # 收集每个节点名出现次数及编号
        for item in nodes:
            m = node_pat.match(item)
            if not m:
                # 结构不符也算一个惩罚，但不立刻 append
                score -= 2
                continue
            name = m.group('name')
            num  = m.group('num')
            counts[name] += 1
            nums[name].add(int(num) if num else 0)

        # 针对这个 guess，统一计算一个 score
        for name, cnt in counts.items():
            seen = nums[name]
            if cnt == 1:
                if seen != {0}:
                    score -= 2
            else:
                required = set(range(1, cnt+1))
                if seen != required:
                    score -= 2
            score += 0.5

        scores.append(score)

    return scores


<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [27]:
# max_prompt_length = maximum_length + 1 # + 1 just in case!
# max_completion_length = max_seq_length - max_prompt_length
max_prompt_length = 4096
max_completion_length = 4096

from vllm import SamplingParams
vllm_sampling_params = SamplingParams(
    min_p = 0.1,
    top_p = 1.0,
    top_k = -1,
    seed = 3407,
    stop = [tokenizer.eos_token],
    include_stop_str_in_output = True,
)

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    vllm_sampling_params = vllm_sampling_params,
    temperature = 1.0,
    learning_rate = 5e-6,
    weight_decay = 0.01,
    warmup_ratio = 0.1,
    lr_scheduler_type = "linear",
    optim = "adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 4, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_completion_length,
    num_train_epochs = 6, # Set to 1 for a full training run
    # max_steps = 1000,
    save_steps = 100,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [28]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        check_answer,
        check_node_label,
        match_format_exactly
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 50 | Num Epochs = 6 | Total steps = 300
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 87,293,952/8,278,029,312 (1.05% trained)


Step,Training Loss,reward,reward_std,completion_length,kl,rewards / check_answer,rewards / check_node_label,rewards / match_format_exactly
1,0.031100,-1.625000,1.600781,468.000000,0.777684,-5.000000,3.375000,0.000000
2,0.041400,6.000000,0.000000,62.000000,1.036199,5.000000,1.000000,0.000000
3,0.019500,6.500000,0.000000,440.500000,0.487431,5.000000,1.500000,0.000000
4,0.033600,0.436688,1.815284,232.750000,0.838931,-2.188312,2.625000,0.000000
5,0.017700,-2.535714,4.279559,319.000000,0.441497,-2.285714,-0.250000,0.000000
6,0.053700,5.500000,0.000000,122.250000,1.342580,5.000000,0.500000,0.000000
7,0.042400,2.488636,1.141730,372.500000,1.059592,-0.136364,2.625000,0.000000
8,0.062500,-4.000000,0.000000,48.000000,1.561919,-5.000000,1.000000,0.000000
9,0.051900,3.500000,0.000000,118.750000,1.296596,3.000000,0.500000,0.000000
10,0.044400,4.000000,0.000000,69.000000,1.110853,3.000000,1.000000,0.000000


TrainOutput(global_step=300, training_loss=0.04039203162305057, metrics={'train_runtime': 2233.136, 'train_samples_per_second': 0.134, 'train_steps_per_second': 0.134, 'total_flos': 0.0, 'train_loss': 0.04039203162305057})

<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [29]:
system_prompt = \
f"""You are a helpful linguist, especially expertise in English etymology.
The user will give you a piece of text that describes etymology of a word, you must output its structured etymology information in the given pattern.
Always output ONLY the structured information. No extra words.
Always remember, in side the output, etymology structure should be placed between {structure_start} and {structure_end}, and detailed content of each node should be placed between {content_start} and {content_end}.
If the input is not relevant to etymology, you should output CANNOT_RECOGNISE."""

text = system_prompt + """
Summary
Formed within English, by derivation.
< efficient adj. + ‑ly suffix2.
"""

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

'< base: efficient adj.. < Latin efficax adj. ‘effectual, productive’, from efficere ‘make effective’, literally, ‘do at home’, from e- ‘out, from’1 + facere ‘make’.\n< base: facere. < Latin base *bāgh2, < Indo-European base *bhag- ‘split, break, join asunder, scatter apart’, < *bhag ‘part, share’, of pre-Indo-European origin. >\n</structure>\nefficiently n.\nn. & adv. 2. as or in the manner of an efficient.\n\nThe structured etymology information for "efficiently" based on your input would be:\n\n<structure>\n<efficiently n. & adv.>\n<2. as or in the manner of an efficient>\n<efficient adj.><<-ly suffix2>\n<base: efficient adj.><<Latin efficax adj. ‘effectual, productive’, from efficere ‘make effective’, literally, ‘do at home’, from e- ‘out, from’1 + facere ‘make’>\n<base: facere><<Latin base *bāgh2, < Indo-European base *bhag- ‘split, break, join asunder, scatter apart’, < *bhag ‘part, share’, of pre-Indo-European origin.>>\n</structure>'

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [30]:
model.save_lora("grpo_saved_lora")

Verify LoRA is actually trained!

In [31]:
from safetensors import safe_open

tensors = {}
with safe_open("grpo_saved_lora/adapter_model.safetensors", framework = "pt") as f:
    # Verify both A and B are non zero
    for key in f.keys():
        tensor = f.get_tensor(key)
        n_zeros = (tensor == 0).sum() / tensor.numel()
        assert(n_zeros.item() != tensor.numel())

Now we load the LoRA and test:

In [32]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": """
Summary
A borrowing from Italian.\nEtymon: Italian maneggiare.
< Italian maneggiare to handle (1298–1309), to be able to use skilfully, to manage, to direct or exercise a horse (14th cent.; > Spanish manejar to manage, use, manipulate (1591)), probably < an unattested post-classical Latin verb < classical Latin manus hand (see manus n.1) + post-classical Latin ‑izare ‑ize suffix. Compare French manier (see manner n.).\nNotes\nRecorded earlier than the cognate manage n.2 Although the earliest attested form in English is manege, the ending was, as in the noun, already in the 16th cent. assimilated to the English ending ‑age. In the late 17th and early 18th cent. the form menage was chiefly used where the sense was close to that of Middle French, French †mesnager (French ménager: see mesnage v.; 1567 in sense ‘to administer’, 1570 in senses ‘to spare’, and ‘to arrange with care’, 1628–30 in sense ‘to treat with consideration’). It is also likely that this French verb influenced the sense-development of the English word: in the writings of Dryden and his contemporaries, there are frequent uses of manage (as well as the form menage) which appear to be consciously modelled on the French word.
"""},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = False,
)
from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 2048,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

'Compare Catalan managejar to manage (1558 as †manaxar), Portuguese manejar (15th cent.), Galician maneixar (1547).\n<structure>\nEnglish - Italian - (Post-Classical Latin 1 + Post-Classical Latin 2) - Classical Latin\n</structure>\n<content>\n@English: {{manage}}\n@Italian: {{maneggiare}} [to handle ({$time(1298–1309)}), to be able to use skilfully, to manage, to direct or exercise a horse ({$time(14th cent.)})] ((> @Spanish {{manejar}} [to manage, use, manipulate] ({$time(1591)}) ))\n@Post-Classical Latin 1: {{*}} [(unattested post-classical Latin verb)]\n@Post-Classical Latin 2: {{‑izare}} [‑ize suffix]\n@Classical Latin: {{manus}} [hand] ((see manus n.1))\n</content>\n'

In [33]:
# test set inference

import re

def extract_and_format_content(input_str: str) -> str:
    """
    Extract contents in <structure> and <content> tags, and return a new formatted string
    """
    input_str = input_str.split("<|endoftext|>")[1]
    structure_match = re.search(r"<structure>(.*?)</structure>", input_str, re.DOTALL)
    s_content = structure_match.group(1) if structure_match else ""

    content_match = re.search(r"<content>(.*?)</content>", input_str, re.DOTALL)
    c_content = content_match.group(1) if content_match else ""

    new_string = f"<structure>\n{s_content}\n</structure>\n<content>\n{c_content}\n</content>"

    return new_string


from unsloth.chat_templates import get_chat_template
import json
from time import time

with open('test_set_small.jsonl', 'r', encoding='utf-8') as f_in, open('inference_grpo.jsonl', 'w', encoding='utf-8') as f_out:

    eta = []
    for line in f_in:
        data = json.loads(line)
        messages = [
            {
              "role": "system",
              "content": system_prompt
            },
            {
              "role": "user",
              "content": data['messages'][1]["content"]
            }
        ]
        # print(messages)


        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt = True, # Must add for generation
        )

        t_start = time()
        output = model.fast_generate(
            text,
            sampling_params = sampling_params,
            lora_request = model.load_lora("grpo_saved_lora"),
        )[0].outputs[0].text
        t_end = time()
        eta.append(t_end - t_start)

        output = [{}, {}, {"role": "assistant", "content": output}]
        json.dump({"messages": output}, f_out)
        f_out.write('\n')
        print("Proceeded line")

print(f"ETA: {sum(eta)/len(eta)}")

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Proceeded line


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Proceeded line


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

...

Proceeded line


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Proceeded line


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Proceeded line
ETA: 3.3151535987854004


Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!